<a href="https://colab.research.google.com/github/abdulrah77/python/blob/main/youtube_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Importing Libraries**

In [ ]:
#import libraries
from IPython.display import JSON
import pandas as pd
from googleapiclient.discovery import build
import time
import requests

In [ ]:
!pip install --upgrade google-api-python-client --quiet

In [ ]:
#Keys
API_KEY ='*********'
channel_id = 'UCW8Ews7tdKKkBT6GdtQaXvQ'

**Defining Funcions**

Get Video details using Youtube API

In [ ]:
def get_videos(df):
  pageToken =""
  url = "https://www.googleapis.com/youtube/v3/search?key=" +API_KEY+"&channelId="+channel_id+"&part=snippet,id&order=date&maxResults=1000"+pageToken
  response = requests.get(url).json()
  time.sleep(1)

  for video in response['items']:
    if video['id']['kind']== "youtube#video":
      video_id= video['id']['videoId']
      video_title=video['snippet']['title']
      #upload_date = str(video['snippet']['publishedAt']).split("T")[0]

      view_count,like_count, comment_count = get_video_details(video_id)
      #save data in pandas df
      df=df.append({'video_id':video_id,'video_title':video_title, 'view_count':view_count,'like_count':like_count, 'comment_count':comment_count  }, ignore_index=True)
      
  return df

In [ ]:
def get_video_details(video_id):
  url_video="https://www.googleapis.com/youtube/v3/videos?id="+video_id+"&part=statistics&key="+API_KEY
  response = requests.get(url_video).json()
  view_count=response['items'][0]['statistics']['viewCount']
  like_count=response['items'][0]['statistics']['likeCount']
  # dislike_count=
  comment_count=response['items'][0]['statistics']['commentCount']

  return view_count,like_count, comment_count

In [ ]:
# Creating the DataFrame
df = pd.DataFrame(columns=["video_id","video_title","view_count","like_count","comment_count"])

df = get_videos(df)

In [ ]:
df.head()
#Check why the two upload_date columns are created?


,video_id,video_title,view_count,like_count,comment_count
0,jN5hpgBrz8k,Dealing With Python Date Manipulations In a Me...,1425,52,7
1,yFyyP_Uvxj0,How To Solve Data Science Probability Intervie...,1635,56,2
2,Av92CaPTRr8,How to use SQL window functions: RANK vs DENSE...,2055,60,8
3,VEjxlKBkZGM,Amazon Data Science Interview Question Walkthr...,5365,171,20
4,hMUf7DqG1nQ,SQL Basics: How to Join Multiple Tables in SQL...,3075,90,6


In [ ]:
#del df['upload_date']

In [ ]:
!pip install mysql.connector

In [ ]:
!pip install PyMySQL

Uploading DataFrame to the MySQL Database 

In [ ]:
#importing my sql connector
# import mysql.connector as mc
import pymysql as ps

Connecting to the Database

In [ ]:
def connect(hostname,port,username, password):
  try:
    conn = ps.connect(host=hostname , port= port, user= username, password=password)
  except ps.OperationalError:
    raise ps.OperationalError
  else:
    print('Connected!')
  return conn

In [ ]:
#update row if video exists
def update_row(curr,video_title, view_count ,like_count, comment_count, video_id):
  query= ('''UPDATE aakash_youtube
             SET video_title =%s,
                 view_count =%s,
                 like_count =%s,
                 comment_count =%s,
             WHERE video_id == %s;,
          ''')
  var_to_update =( video_title, view_count ,like_count, comment_count, video_id )
  curr.execute(query,(var_to_update))

In [ ]:
#Creating a table
def create_table(curr):
  query =('''CREATE TABLE IF NOT EXISTS aakash_youtube (
            
            video_id VARCHAR(255) PRIMARY KEY,
            video_title TEXT NOT NULL,
            view_count INTEGER NOT NULL,
            like_count	INTEGER NOT NULL,
            comment_count INTEGER NOT NULL)

            ''')
  curr.execute(query)

In [ ]:
#Check to see if video exists
def check_if_video_exists(curr, video_id):
  query = (""" SELECT video_id FROM aakash_youtube WHERE video_id = %s""")
  curr.execute(query,(video_id,))

  return curr.fetchone() is not None

In [ ]:
#Check to see if video exists
def update_db(curr,df):
  tmp_df = pd.DataFrame(columns=['video_id', 'video_title', 'view_count', 'like_count', 'comment_count'])
  print(tmp_df.columns.tolist())
  for i,row in df.iterrows():
    if check_if_video_exists(curr,row['video_id']):
        update_row(curr, row['video_id'],row['video_title'], row['view_count '],row['like_count'], row['comment_count'])
    else:
        tmp_df = tmp_df.append(row)
        #print(i)
        #print(row)
        pass
  return tmp_df

In [84]:
#write insert function
def insert_into_table(curr, video_id, video_title, view_count, like_count, comment_count):
  print("check1")
  if (view_count is None):
    view_count = 0
  elif (like_count is None):
    like_count = 0
  elif (comment_count is None):
    comment_count =0
  else:
    insert_into_videos = (""" INSERT INTO aakash_youtube (video_id,video_title,view_count,like_count,comment_count) VALUES(%s,%s,%s,%s,%s);""")
    print("check2")
    row_to_insert = (video_id, video_title, view_count, like_count, comment_count)
    print("check3")
    print(curr.execute(insert_into_videos, row_to_insert))

In [86]:
def append_from_df_to_db(curr,df):
  print(df.columns.tolist())
  for i,row in df.iterrows():
    insert_into_table(curr,row.get('video_id'),row.get('video_title'), row.get('view_count '),row.get('like_count'), row.get('comment_count'))
    print(i)

In [ ]:
#storing database connection details
hostname = 'database-2.cimnlngca8v0.ap-south-1.rds.amazonaws.com'
dbname = 'database-2'
port = 3306
username = 'admin'
password = 'mypassword'
conn = None

In [ ]:
conn = connect(hostname,port,username,password)
curr = conn.cursor()

Connected!


In [ ]:
query = ''' CREATE DATABASE IF NOT EXISTS youtubedata'''

In [ ]:
curr.execute(query)

1

In [ ]:
# conn.cursor().execute("select version()")

In [ ]:
query = '''USE youtubedata'''
curr.execute(query)


0

In [ ]:
curr.connection.commit()

In [ ]:
curr.execute("select version()")
print(curr.fetchone())

('8.0.27',)


In [ ]:
conn.commit()

In [ ]:
# df.columns.tolist()


Check if Table has been created

In [ ]:
create_table(curr)

In [ ]:
# query= 'show tables'
# curr.execute(query)
# curr.fetchall()

(('aakash_youtube',),)

In [68]:
new_vid_df = update_db(curr,df)

['video_id', 'video_title', 'view_count', 'like_count', 'comment_count']


In [ ]:
df.head()

In [ ]:
new_vid_df.head()

In [ ]:
append_from_df_to_db(curr,new_vid_df)

In [88]:
conn.commit()